# Введение в обработку естественного языка

## Курсовая работа

In [1]:
!pip install pymorphy2[fast] annoy stop_words transformers

     |████████████████████████████████| 55 kB 1.6 MB/s 
     |████████████████████████████████| 646 kB 5.4 MB/s 
     |████████████████████████████████| 2.9 MB 37.9 MB/s 
     |████████████████████████████████| 596 kB 48.7 MB/s 
     |████████████████████████████████| 895 kB 39.8 MB/s 
     |████████████████████████████████| 56 kB 4.5 MB/s 
     |████████████████████████████████| 3.3 MB 43.9 MB/s 
     |████████████████████████████████| 8.2 MB 13.9 MB/s 
     |████████████████████████████████| 371 kB 61.4 MB/s 
  Created wheel for annoy: filename=annoy-1.17.0-cp37-cp37m-linux_x86_64.whl size=391678 sha256=4eec72a8a2e90c15e7d1ecacaafbd5439af04e32da75968edc4a80e4f77f6529
  Stored in directory: /root/.cache/pip/wheels/4f/e8/1e/7cc9ebbfa87a3b9f8ba79408d4d31831d67eea918b679a4c07
  Created wheel for stop-words: filename=stop_words-2018.7.23-py3-none-any.whl size=32912 sha256=9fe4df0557b6d4af9e4f4b2b7cf4b9c6e53df72dca6bb73255361a6277d4aa4d
  Stored in directory: /root/.cache/pip/wheels/fb/86/

In [3]:
import string
import nltk
import annoy
import numpy as np
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
from tqdm.notebook import tqdm
from transformers import AutoTokenizer, AutoModel
from datetime import datetime
import pickle
import os

import nltk
nltk.download('stopwords')

from google.colab import drive
drive.mount('/content/drive')

path = '/content/drive/MyDrive/Colab Notebooks/data/gb_NLP_cv/'

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
question = None
written = False

path_file = '/content/drive/MyDrive/Colab Notebooks/data/gb_NLP_cv/Otvety.txt'

with open("prepared_answers.txt", "w") as fout:
    with open(os.path.join(path, 'Otvety.txt'), "r") as fin:
        for line in tqdm(fin):
            if line.startswith("---"):
                written = False
                continue
            if not written and question is not None:
                fout.write(question.replace("\t", " ").strip() + "\t" + line.replace("\t", " "))
                written = True
                question = None
                continue
            if not written:
                question = line.strip()
                continue

0it [00:00, ?it/s]

In [5]:
def preprocess_txt(line):
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return ' '.join(spls)

In [6]:
morpher = MorphAnalyzer()
sw = set(get_stop_words("ru") + nltk.corpus.stopwords.words('russian'))
exclude = set(string.punctuation)

In [7]:
tokenizer = AutoTokenizer.from_pretrained("sberbank-ai/sbert_large_nlu_ru")
model = AutoModel.from_pretrained("sberbank-ai/sbert_large_nlu_ru")

Downloading:   0%|          | 0.00/323 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/655 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.70M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.59G [00:00<?, ?B/s]

In [8]:
q_len = []
a_len = []
with open("prepared_answers.txt", "r") as f:
    counter = 0
    for line in tqdm(f):
        spls = line.split("\t")
        q_len.append(len(spls[0].split()))
        a_len.append(len(spls[1].split()))

0it [00:00, ?it/s]

In [9]:
np.median(q_len), np.mean(q_len), np.quantile(q_len, 0.75), np.quantile(q_len, 0.85), np.quantile(q_len, 0.95), np.max(q_len)

(12.0, 20.454707214215595, 21.0, 31.0, 63.0, 862)

In [12]:
bert_index = annoy.AnnoyIndex(1024 , 'angular')

start = datetime.now()
index_map = {}
counter = 0
with open("prepared_answers.txt", "r") as f:
    for line in tqdm(f):
        spls = line.split("\t")
        if len(spls[0].split()) < 2 or len(spls[1].split()) < 3 or len(spls[0].split()) > 15:
            continue
        
        index_map[counter] = spls[1]
        question = preprocess_txt(spls[0])
        
        tok = tokenizer(question, padding=True, truncation=True, max_length=24, return_tensors='pt')
        out_state = model(**tok)[1].detach().numpy()[0]
        bert_index.add_item(counter, out_state)
        counter += 1

        diff = datetime.now() - start
        if diff.seconds > 0 and diff.seconds % 3600 == 0:
          print(diff, counter)
        
        if diff.seconds >= 28800: #42000
          bert_index.build(10)
          bert_index.save(os.path.join(path, 'bert_index'))
          with open(os.path.join(path, 'index_map.pkl'), 'wb') as file:
            pickle.dump(index_map, file)
          break

0it [00:00, ?it/s]

1:00:00.330173 12221
1:00:00.625587 12222
1:00:00.922258 12223
2:00:00.064080 24509
2:00:00.294791 24510
2:00:00.641709 24511
2:00:00.895955 24512
3:00:00.114742 36865
3:00:00.346389 36866
3:00:00.661377 36867
3:00:00.963201 36868
4:00:00.057872 49133
4:00:00.368919 49134
4:00:00.599220 49135
4:00:00.851650 49136
5:00:00.179670 61317
5:00:00.484065 61318
5:00:00.779939 61319
6:00:00.060260 73835
6:00:00.335544 73836
6:00:00.680931 73837
7:00:00.120237 86208
7:00:00.338952 86209
7:00:00.579879 86210
7:00:00.868327 86211
8:00:00.024203 98186


In [13]:
def get_response(question, index, index_map):
    question = preprocess_txt(question)
    tok = tokenizer(question, padding=True, truncation=True, max_length=24, return_tensors='pt')
    vector = model(**tok)[1].detach().numpy()[0]
    answers = index.get_nns_by_vector(vector, 2)
    return [index_map[i] for i in answers]

In [21]:
get_response("как вас зовут?", bert_index, index_map)

['Ринат Леонидович Ахметов <br>Рина́т Леони́дович Ахме́тов — донецкий предприниматель, промышленник, миллиардер, самый богатый человек ...Википедия <br>Родился: 21 сентября 1966 г. (46 лет) , Донецк, Украина <br>Образование: Донецкий национальный университет <br>Политическая партия: Партия регионов. \n',
 'этот парень с группы ленинград со шнуром вместе играл. \n']

In [23]:
get_response("какой фильм посмотреть?", bert_index, index_map)

['зависит от вкусов конечно) хотите тяжёлый и грустный фильм-посмотрите Бен Х или Куда приводят мечты например... в фильмах немного разбираюсь) если что нужно вдруг-пишите. \n',
 'ЛОЛ- лето одноклассники любовь!!! Офигенный фильм!!!. \n']